In [ ]:
import earthaccess
import xarray as xr
import dask
import numpy as np

In [5]:
auth = earthaccess.login(strategy='interactive', persist = True)

In [6]:
# search for results (testing with just a couple days)

results = earthaccess.search_data(
    short_name='NSIDC-0051',
    temporal=('2021-11-01', '2021-12-01'),
    bounding_box=(-180, 0, 180, 90)
)


In [20]:
# coastal mask function in next two cells

import geopandas as gpd
import cartopy.feature as cfeature
from rasterio import features
from scipy.ndimage import convolve

In [ ]:
def select_coastal(ds):

    # load land polygons and reproject to EPSG:3411

    land = gpd.read_file("data/naturalearth/ne_110m_admin_0_countries.shp")
    land = land.to_crs(epsg=3411)

    # get transform for rasterizing

    dx = float(ds.x.diff('x').mean())  # 25000 meters
    dy = float(ds.y.diff('y').mean())  # 25000 meters
    x0 = float(ds.x.min())
    y0 = float(ds.y.min())
    transform = [dx, 0, x0, 0, -dy, y0]

    # rasterize land mask: 1 = land, 0 = ocean

    land_mask = features.rasterize(
        ((geom, 1) for geom in land.geometry),
        out_shape=(ds.sizes['y'], ds.sizes['x']),
        transform=transform,
        fill=0,
        dtype=np.uint8
    )

    # create coastal mask (within 3 grid cells of land)

    ocean = (land_mask == 0).astype(int)
    kernel = np.ones((7, 7))  # 3-cell radius
    land_neighbor_count = convolve(1 - ocean, kernel, mode='constant', cval=0)
    coastal_mask = (ocean == 1) & (land_neighbor_count > 0)

    # convert to xarray.DataArray

    coastal_mask_xr = xr.DataArray(
        coastal_mask,
        coords={'y': ds.y, 'x': ds.x},
        dims=('y', 'x')
    )

    # apply mask

    return ds.where(coastal_mask_xr)


In [22]:
# open results with xarray

files = earthaccess.open(results)
ds = xr.open_mfdataset(files, parallel=True, combine='by_coords', preprocess=select_coastal)

QUEUEING TASKS | :   0%|          | 0/66 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/66 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/66 [00:00<?, ?it/s]

AttributeError: 'NaturalEarthFeature' object has no attribute 'path'

In [12]:
ds

<xarray.Dataset> Size: 34MB
Dimensions:     (time: 31, y: 448, x: 304)
Coordinates:
  * x           (x) float64 2kB -3.838e+06 -3.812e+06 ... 3.712e+06 3.738e+06
  * y           (y) float64 4kB 5.838e+06 5.812e+06 ... -5.312e+06 -5.338e+06
  * time        (time) datetime64[ns] 248B 2021-11-01 2021-11-02 ... 2021-12-01
Data variables:
    crs         (time) |S1 31B b'' b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    F17_ICECON  (time, y, x) float64 34MB dask.array<chunksize=(1, 448, 304), meta=np.ndarray>
Attributes: (12/49)
    title:                     Sea Ice Concentrations from Nimbus-7 SMMR and ...
    summary:                   This data set is generated from brightness tem...
    id:                        10.5067/MPYG15WAA4WX
    license:                   Access Constraint: These data are freely, open...
    acknowledgment:            These data are produced by the NASA Cryospheri...
    metadata_link:             https://doi.org/10.5067/MPYG15WAA4WX
    ...                        ...
    geospatial_lat_units:      degrees_north
    geospatial_lon_units:      degrees_east
    product_version:           v2.0
    source:                    Polar stereographic brightness temperatures fr...
    instrument:                SSMIS > Special Sensor Microwave Imager/Sounder
    platform:                  DMSP 5D-3/F17 > Defense Meteorological Satelli...

In [ ]:
# mapping and some data analysis

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [ ]:
ice = ds['F17_ICECON'].isel(time = 0)

ax = plt.axes(projection=ccrs.NorthPolarStereo())
ax.coastlines()

ice.plot(
    ax=ax,
    transform=ccrs.NorthPolarStereo(),
    cmap='Blues',
    cbar_kwargs={'label': 'Sea Ice Concentration (%)'}
)

# ax.coastlines()
# ax.add_feature(cfeature.BORDERS, linestyle=':')
# ax.set_extent([-180, 180, 50, 90], crs=ccrs.PlateCarree())

plt.show()

In [ ]:
x = ds['x']
y = ds['y']

# create 2D grids of x and y
X, Y = np.meshgrid(x, y)

# plot
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.NorthPolarStereo())

# This is the critical fix:
# use pcolormesh with the source projection
pcm = ax.pcolormesh(X, Y, ice, transform=ccrs.epsg(3411), cmap='Blues', shading='auto')

# add coastlines and features
ax.coastlines()
ax.add_feature(cfeature.LAND, zorder=-100, edgecolor='black')
ax.set_extent([-180, 180, 50, 90], crs=ccrs.PlateCarree())

# colorbar
plt.colorbar(pcm, ax=ax, orientation='vertical', label='Sea Ice Concentration (%)')

# save before show
plt.savefig("seaice_plot.png")
plt.show()